In [1]:
%autosave 120


Autosaving every 120 seconds


In [120]:
import pandas as pd
import xlrd
import numpy as np
import openpyxl as opx
import glob
import sys
import datetime
import xlwings as xl
import copy
from openpyxl.utils import range_boundaries
from fuzzywuzzy import fuzz
import pyodbc
import math as math
path = 'C:\\trial'
ticker_list =[] 
folder = glob.glob(path + '\\*')
company = [1]
import ast
c = 0
import configparser
con=configparser.ConfigParser()
con.read("C:\\Users\\admin\\thapovan\\Thapovan2\\Config.cfg")

['C:\\Users\\admin\\thapovan\\Thapovan2\\Config.cfg']

In [96]:
import pandas as pd
import os
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import PorterStemmer
stemmer = PorterStemmer()
import numpy as np
np.random.seed(2018)
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
lem = WordNetLemmatizer()
stem = PorterStemmer()
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
stop_words = set(stopwords.words("english"))
new_words = ["using", "show", "result", "large", "also", "iv", "one", "two", "new", "previously", "shown"]
stop_words = stop_words.union(new_words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [100]:
os.chdir("C:\\Users\\admin\\thapovan\\Thapovan2\\Corpus")

In [106]:
data1 = pd.read_csv("values1.csv", error_bad_lines=False)
data = pd.read_csv("Thapovan_data.csv", error_bad_lines=False)

In [97]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [98]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [116]:
def return_match_list(db,trial):
    p_list = []
    for i in range(0,len(trial)):
        metric = trial['trial'][i]
        text = re.sub('[^a-zA-Z]', ' ', metric)
    
        #Convert to lowercase
        text = text.lower()
    
        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)
    
        ##Convert to list from string
        text = text.split()
    
        ##Stemming
        ps=PorterStemmer()
        #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
        text = " ".join(text)
        tex_check = text.split(" ")
        list=[]
        for key in data1["Keyword"]:
            count = 0
            for j in tex_check:
                if(j in key):
                    count = count+1
            list.append((count/len(tex_check))*100)  
        l = []
        count1 = 0 
        maxi = max(list)
        for val in list:
            if(val==maxi):
                l.append(count1)
            count1 = count1+1 
        l2=[]
        for j in l:
            l2.append(data['Term'][j])
        p_list.append(l2)      
    trial['possible'] = p_list
    t_list = []
    for i in range(0,len(db)):
        metric = db['metric_name'][i]
        text = re.sub('[^a-zA-Z]', ' ', metric)
    
        #Convert to lowercase
        text = text.lower()
    
        #remove tags
        text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
        # remove special characters and digits
        text=re.sub("(\\d|\\W)+"," ",text)
    
        ##Convert to list from string
        text = text.split()
    
        ##Stemming
        ps=PorterStemmer()
        #Lemmatisation
        lem = WordNetLemmatizer()
        text = [lem.lemmatize(word) for word in text if not word in  
            stop_words] 
        text = " ".join(text)
        tex_check = text.split(" ")
        list=[]
        for key in data1["Keyword"]:
            count = 0
            for j in tex_check:
                if(j in key):
                    count = count+1
            list.append((count/len(tex_check))*100)  
        l = []
        count1 = 0 
        maxi = max(list)
        for val in list:
            if(val==maxi):
                l.append(count1)
            count1 = count1+1 
        l2=[]
        for j in l:
            l2.append(data['Term'][j])
        t_list.append(l2)    
    db['possible'] = t_list    
    final_list = []
    fill = []
    for i in range(0,len(trial)):
        metric = trial['possible'][i]
        final = []
        for j in range(0,len(db)):
            count = 0
            for k in db['possible'][j]:
                if(k in metric):
                    count = count+1
            final.append((count/len(db['possible'][j]))*100)
            l3 =[]
            for k in range(0,len(final)):
                if(final[k]>0):
                    l3.append(db['metric_name'][k])
        if(len(l3)==len(final)):
            final_list.append(fill)
        else:    
            final_list.append(l3) 
    return(final_list)  

In [117]:
def ticker_dictionary_build(tick):
    db_metrics = input("enter the name of the excel sheet with the database metrics for this particular ticker")
    analyst_metrics = input("enter the name of the excel sheet with the analyst metrics for this particular ticker")
    db = pd.read_csv(db_metrics, error_bad_lines=False)
    trial = pd.read_csv(analyst_metrics, error_bad_lines=False)
    final_matches = return_match_list(db,trial)
    trial['matches'] = final_matches
    csv_name = tick + "_matches.csv"
    trial.to_csv(csv_name)
    return()

In [110]:
#connection to database:
server = ast.literal_eval(con.get("Setting","server"))
database = ast.literal_eval(con.get("Setting","database"))
username = ast.literal_eval(con.get("Setting","username"))
password = ast.literal_eval(con.get("Setting","password"))
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [5]:

data = []
#function which get information of file using its path and return 5 elements(year, month,date,ticker code,analyst name)
def formating(filen):
    data = filen.replace(path, '').split('\\')
    del data[0]
    del data[3]
    if '-' in data[-1]:
        temp = (data[-1].split('.')[0]).split('-')
    elif '_' in data[-1]:
        temp = (data[-1].split('.')[0]).split('_')
    else:
        temp = (data[-1].split('.')[0]).split(' ')
    del data[-1]
    data.append(temp[0])
    data.append(temp[1])
    print(data)
    return(data)

In [56]:

#defined variable which help us to recognize year and quater of given column

year_pattern_int=[]
year_pattern_float=[]
year_pattern_2di=[]
quater_pattern=ast.literal_eval(con.get("Setting","quater_pattern"))
bscf_names=ast.literal_eval(con.get("Setting","bscf_names"))
#bscf_names=["balancesheet","cashflows","bs&cf","cf&bs","bsheet","balancesheet&cashflow","cashflow&balance sheet","cash&balance","adjustedbs","annualbs&cf","annualcf&bs"]

check_list = ast.literal_eval(con.get("Setting","check_list"))
for i in range(2000,2031):
    year_pattern_int.append(i)
    year_pattern_float.append(float(i))
    if i-2000<10:
        year_pattern_2di.append('0'+str(i-2000))
    else:
        year_pattern_2di.append(str(i-2000))


In [114]:
# type of main function

def function1(excel_file,path_list):
    #read excel file using openpyxl
    wb=opx.load_workbook(excel_file,data_only=True)
    sheet_tobechecked=wb.sheetnames #getting all the sheet name
    nonhidden=[]
    startrow={}
    print(sheet_tobechecked)
    #this part helps to ignore hidden sheets and to take related to BS , Cf and IS
    names=[]
    for i in range(0, len(sheet_tobechecked)):
        curr_sheet = wb[sheet_tobechecked[i]]
        if curr_sheet.sheet_state != 'hidden':
            nonhidden.append(sheet_tobechecked[i])
            if sheet_tobechecked[i].lower()=='bs' or sheet_tobechecked[i].lower()=='cf' or sheet_tobechecked[i].lower()=='model':
                names.append(sheet_tobechecked[i])
            else:
                count_nobscf=1
                for nam in bscf_names:
                    if fuzz.partial_token_set_ratio(sheet_tobechecked[i].lower(),nam)>=80:
                        names.append(sheet_tobechecked[i])
    
    sheet_tobechecked = names # all the sheet which are important to us
    
    #if no sheet found then return giving error as sheet_name_fault
    if len(sheet_tobechecked)<=0:
        return (2) # that means there is problem
        '''print(nonhidden)
        while True:
            sheee=input("Enter the sheet name U want to process from above or for exit input'e' ")
            if sheee!='e':
                sheet_tobechecked.append(sheee)
            else:
                break'''
    
    sheet_tobechecked=list(set(sheet_tobechecked))# to remove repeated sheet names
    print(sheet_tobechecked)
    print("sheet to be checked")
        
    #defining variable
    year_pre,quater_pre,q4_check_pre='','',0
    
    #unmerging all cells of every sheet of a file
    for every_sheet in sheet_tobechecked:
        
        startrow_no=startingrow(wb,every_sheet)-1
        if startrow_no<0:
            startrow[every_sheet]=0
        else:
            startrow[every_sheet]=startrow_no
        
        curr_sheet=wb[every_sheet]
        merge_list=copy.deepcopy(curr_sheet.merged_cells.ranges)
        for group in merge_list :
            split_arr = group.coord.split(':')
            min_col, min_row, max_col, max_row = range_boundaries(group.coord)
            top_left_cell_value = curr_sheet[split_arr[0]].value
            curr_sheet.unmerge_cells(group.coord)
            for row in curr_sheet.iter_rows(min_col=min_col, min_row=min_row, max_col=max_col, max_row=max_row):
                for cell in row:
                    cell.value = top_left_cell_value
    wb.save(excel_file) #the modified file is been overwritten (have to look over it afterward by changing path)
    print("excel")
    display(excel_file)
    for sheetno in range(0,len(sheet_tobechecked)):
        print(sheet_tobechecked[sheetno])
        address_dataframe=pd.DataFrame(columns=['q1','q2','q3','q4','tl','Ex']) # defining lookup table
        excel_data = pd.read_excel(excel_file,sheet_name=sheet_tobechecked[sheetno],header=None,skiprows=[i for i in range(0,startrow[sheet_tobechecked[sheetno]])])
        pd.options.display.max_columns = None
        pd.options.display.max_rows=None
        print("This is excel data")
        display(excel_data)
        
        #formating the dataframe by removing column and row having less than 10 element
        excel_data=excel_data.replace(0,np.nan)
        excel_data=excel_data.dropna(axis=0,thresh=10)
        excel_data=excel_data.dropna(axis=1,thresh=10)
        excel_data=excel_data.reset_index(drop=True)
        if(len(excel_data.columns)==0):
            continue # this line was added to handle exceptions in the code
        excel_data.columns = range(excel_data.shape[1])
        pd.options.display.max_columns = None
        pd.options.display.max_rows=None
        display(excel_data)
        startrowno=0
        
        #getting defination of each column
        print(len(excel_data.columns))
        if len(excel_data.columns)>10: #changed from 10 to 3
            temp_dataframe=excel_data[:10] # the first 10 element of a column will tell us about it 
            for j in temp_dataframe:
                list_ele=list(temp_dataframe[j])
                knowledge=[]
                colno=j
                for i in list_ele:
                    #overlook element if its nan,none, or weeks or days written in it
                    if str(i)=='nan' or str(i)=='None' or type(i)==datetime.datetime or "week" in str(i) or "day" in str(i):
                        a=1
                    #take only element which are keywords to us (like FY2017,3Q17 etc.) or else break which means any value have been occured
                    elif (type(i)==float and i not in year_pattern_float) or (type(i)==int and i not in year_pattern_int):
                        startrowno=list_ele.index(i)
                        break
                    else:
                        knowledge.append(i)
                
                # proces further if the list taken for defining the row is not empty 
                if len(knowledge)>0:
                    
                    year_got,quater_got,q4_check_got=about_q_year(knowledge)
                        
                    #here is a small assumtion (if we got Q4 and all the elementused to define column is same we will take the curent row as Total of particuar year) )
                    if q4_check_got==1 and year_got==year_pre and quater_got==quater_pre and q4_check_got==q4_check_pre:
                        year_final=year_got
                        quater_final='TL'
                        q4_check_final=1
                    else:
                        # or else everything will be consider as it was given has been found about it
                        year_pre=year_got
                        quater_pre=quater_got
                        q4_check_pre=q4_check_got
                        year_final=year_got
                        quater_final=quater_got
                        q4_check_final=q4_check_got                       
                    
                    #if only year is written and nothing about the quater we take that as Total for particular year
                    if year_got!='' and (quater_got=='' or quater_got=='TL' or quater_pre=='TL' or quater_pre==''):
                        quater_final='TL'
                    
                    #printing and making dataframe(lookup table) of information of each columns
                    if year_final!='' and quater_final!='':
                        got_cor=1 #means we got something
                        #print(knowledge,"===",end="")
                        #print("belongs to "+"20"+year_final," "+quater_final)
                        if '1' in quater_final:
                            qval='q1'
                        elif '2' in quater_final:
                            qval='q2'
                        elif '3' in quater_final:
                            qval='q3'
                        elif '4' in quater_final:
                            qval='q4'
                        else:
                            qval='tl'
                        try:
                            address_dataframe.loc["20"+year_final][qval]=[startrowno,colno]
                        except Exception:
                            address_dataframe.loc["20"+year_final]='Nan'
                            address_dataframe.loc["20"+year_final][qval]=[startrowno,colno]                           
                 
        display(address_dataframe)
        #db_connect()
        #extra fro shoeing rows which doesnot match
        not_present=pd.DataFrame(columns=excel_data.columns)
        coll=0
        
        #this is to make a list of year and its respective column number from lookup take which has to consider
        row_name_year=list(address_dataframe.index)
        row_name_year.sort()
        if int(path_list[0])<int(row_name_year[-1]):
            cur_yr=int(path_list[0])
            count_yr=0
            count = 0
            metric_col=[]
            par_year=[]
            metric_row=20
            year_tosearch=cur_yr
            
            lowest_year=min_year("'"+path_list[3]+"'")

            while (count_yr<6 and year_tosearch>=lowest_year):
                year_tosearch=cur_yr-(count_yr+1)
                val=address_dataframe['q1'][str(year_tosearch)]
                if val!='Nan':
                    if val[0]<metric_row:
                        metric_row=val[0]
                    metric_col.append(val[1])
                    par_year.append(year_tosearch)
                    count_yr+=1
            print(par_year)
            print(metric_col)
            print(metric_row)
                

            all_metric=set()
            def_metrics = []
            maybe_metrics = []
            no_metrics = []
            for rowss in range(metric_row,excel_data.shape[0]):
                metric_found=[]
                list_metric_found=[]
                com_met=set()
                a_metric=excel_data.iloc[rowss][0]
                for met_num in range(0,6):
                    #getting the metric name wrt the year,quater,ticker and value send
                    metric=set(get_metric_name(par_year[met_num],path_list[3],excel_data.iloc[rowss][metric_col[met_num]],a_metric))
                    com_met.update(metric)
                    list_metric_found.extend(metric)
                if com_met:
                    print(com_met)
                    print(list_metric_found)
                for un_met in com_met:
                    #if min 60% of them says that yes the values are of  particlar metric we take it 
                    if list_metric_found.count(un_met)/6 >= 0.6:
                        metric_found.append(un_met)
                        def_metrics.append(un_met)
                    elif list_metric_found.count(un_met)/6 >= 0.4 and list_metric_found.count(un_met)/6 <0.6:
                        maybe_metrics.append(un_met)
                    elif list_metric_found.count(un_met)/6 < 0.4:
                        no_metrics.append(un_met)
                if metric_found:
                    print(rowss,metric_found)
                    all_metric.update(metric_found)
                else:
                    not_present.loc[coll]=list(excel_data.loc[rowss])
                    coll+=1
                
            print(all_metric,len(all_metric))#printing all mertric found in database 
            
            metric_db=get_metriclist_from_db(path_list[0],path_list[3])
            print("metric not in actuals---")
            print(list(set(metric_db)-set(all_metric)))# all metric we couldn't find wrt to a ticker
            print("metric not with respect to analyst")
            print(list(set(get_metriclist_wrt_analyst(path_list[3],path_list[4]))-set(all_metric)))#metric which we couldn't finf wrt to analyst
            # the percentge accuracy will be calculate wby no of metric_found by us divided by total number of metric wrt analyst 
            print()
            print()
            print("ACCURACY==")
            print("Number of total element in Actual DB=", len(metric_db))
            print("Number of metric found by us=",len(all_metric))

            print("accuracy wrt to Actual=",len(list(set(metric_db)&set(all_metric)))*100/len(metric_db))
            print()
            print("Number of total element in Estimated DB wrt Analyst=", len(get_metriclist_wrt_analyst(path_list[3],path_list[4])))
            z=set(get_metriclist_wrt_analyst(path_list[3],path_list[4]))
            print("Number of metric found by us=",len(z&set(all_metric)))
            print("Accuracy with respect to estimates=",len(z&set(all_metric))*100/len(z))
            print("list of definate metrics", list(set(def_metrics)))
            print()
            print("list of semi-definate metrics", list(set(maybe_metrics)))
            print()
            print("list of Unmatched metrics", list(set(no_metrics)))
            
            display(not_present)
    
    if got_cor:
        return 0
    else:
        return 1
   
    '''qw=input()
            if qw=='q':
                sys.exit()'''


In [58]:

#get the minimum year present in dtabase for given ticker
def min_year(ticker):
    select_stmt1 = F" select distinct financial_year from dbo.cc_actual_metrics_consolidated where ticker_code={ticker} order by financial_year"
    cursor.execute(select_stmt1)
    row = cursor.fetchone()
    res_array = []
    while row:
        res_array.append(row[0])
        row = cursor.fetchone()
    print(res_array[0])
    return(res_array[0])


In [59]:
def get_metric_name (year,ticker,value,quarter='Q1',a_metric):
    possible_list =[]
    #index = ticker_dictionary[ticker_dictionary['trial']==a_metric].index.item()
    #possible_list = ticker_dictionary['matches'][index]
    #Also write a conditional if statement for the case of an empty list 
    ticker_code = "'"+ticker+"'"
    year = year
    quarter = "'"+quarter+"'"

    if type(value)==int or (type(value)==float and str(value)!='nan'):
        if abs(value)>=1:
            #when value is greater than 1
            valueo= "'"+str(value)+"'" #orginal value
            value1 = "'"+str(int(value))+".%"+"'" # integered value
            value2 = "'"+str(round(value, 2))+"%"+"'" #value when rounded upto 2
            value3 = "'"+str(-1*int(value))+".%"+"'" #integered with negation
            value4 = "'"+str(-1*round(value,2))+"%"+"'" # round upto 2 with negation
            print(valueo,value1,value2,value3,value4)
            select_stmt1 = F"select distinct metric_name from dbo.cc_actual_metrics_consolidated where " \
                F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                F"(value like {valueo} or value like {value1} or value like {value2} or value like {value3} or value like {value4})"

            metric_list=[]
            cursor.execute(select_stmt1)
            row = cursor.fetchone()
            while row:
                
                metric_list.append(row[0])
                row = cursor.fetchone()
        else:
            #if value less than 1
            valueo= "'"+str(value*100)+"'" #orginal with 100 multiplied
            value1 = "'"+str(int(value*100))+".%"+"'" # multiply by 100 and then integered
            value3 = "'"+str(-1*int(value*100))+".%"+"'" # multiply by 100 and integer and negate it
            print("with 100",valueo,value1,value3)
            select_stmt1 = F"select distinct metric_name from dbo.cc_actual_metrics_consolidated where " \
                F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                F"(value like {valueo} or value like {value1} or value like {value3})"
            
            metric_list=[]
            cursor.execute(select_stmt1)
            row = cursor.fetchone()
            flag_got_with_100=0
            while row:
                metric_list.append(row[0])
                row = cursor.fetchone()
                flag_got_with_100=1

            #if not flag_got_with_100:
            valueo = "'"+str(value)+"'" #orginal value
            value1 = "'"+str(round(value,2))+"%"+"'" # round upto 2
            value3 = "'"+str(-1*round(value,2))+"%"+"'" # round upto 2  and negate it
            print("without 100",valueo,value1,value3)
            select_stmt1 = F"select distinct metric_name from dbo.cc_actual_metrics_consolidated where " \
                F"ticker_code={ticker_code} and financial_year = {year} and quarter = {quarter} and " \
                F"(value like {valueo} or value like {value1} or value like {value3})"
            
            cursor.execute(select_stmt1)
            row = cursor.fetchone()
            while row:
                #to_confirm
                metric_list.append(row[0])
                row = cursor.fetchone()

        return(metric_list)
    else:
        return([])


In [60]:
#function which tells about the quater and year the particular column belongs to
def about_q_year(checkar):
    
    checkar_duplicate=np.copy(checkar)
    quater=''
    year=''
    year_flag=0
    quater_flag=0
    q4_check=0
    checkar=[]
    datetime_flag=1
    for i_che in checkar_duplicate:
        checkar.append(str(i_che))
    
    for i_che in range(0,len(checkar)):
        
        if not year_flag:
            #checking if year is presenr in (2017,2016 format)
            for j_y in year_pattern_int:
                if str(j_y) in checkar[i_che]:
                    year=str(j_y)[2:]
                    checkar[i_che]=checkar[i_che].replace(str(j_y),'')
                    checkar[i_che]=checkar[i_che].replace(year,'')
                    year_flag=1
                    break
        if not year_flag:
            #checking if the year is present in 2 digited format like 17,16)
            for j_y in year_pattern_2di:
                if str(j_y) in checkar[i_che]:
                    year=str(j_y)
                    checkar[i_che]=checkar[i_che].replace(year,'')
                    year_flag=1
                    break

        if not quater_flag:
            # getting info about the quater
            for i_p in quater_pattern:
                checkar[i_che]=checkar[i_che].replace(year,'')
                checkar[i_che]=checkar[i_che].replace("20"+year,'')
                if i_p in checkar[i_che].lower():
                    quater=i_p
                    quater_flag=1
                    break       
        
        if year_flag and quater_flag:
            break
    #returning the year and quater and if its 4th quatered or not
    return(year,quater,q4_check)


In [61]:
# funtion to avoid te rough work present in top and retutn the row no from which its important
def startingrow(wb,sheet_name):
    curr_sheet = wb[sheet_name]
    flag = 0
    row = 0
    for col in curr_sheet.iter_cols(max_col=20,max_row=15):
        for cell in col:
            if cell.value is not None:
                for data in check_list:
                    if fuzz.partial_ratio(str(data),str(cell.value)) == 100:
                        row = cell.row
                        flag =1
                        break
                if flag == 1:
                    break
        if flag == 1:
            break

    print(row)
    return(row)

In [62]:
#function which return all the metric anme with repect to ticker in actuals
def get_metriclist_from_db(year, ticker):

    ticker_code = "'"+ticker+"'"
    year = year
    select_stmt1 = F"select distinct metric_name from dbo.cc_actual_metrics_consolidated where ticker_code = {ticker_code} " \
        F" and financial_year = {year}"

    cursor.execute(select_stmt1)
    row = cursor.fetchone()
    metric_list_comp = []
    while row:
        metric_list_comp.append(row[0])
        row = cursor.fetchone()

    return (metric_list_comp)

In [63]:
#function which return aall the metric name wrt to analyst from estimated database
def get_metriclist_wrt_analyst(ticker,analyst):
    ticker_code = "'"+ticker+"'"
    analyst_name="'%"+analyst+"%'"
    select_stmt1 = F" select distinct metric_name from [dbo].[cc_metrics_consolidated] where ticker_code = {ticker_code} and quarter='q1' and analyst_name like {analyst_name} order by metric_name"
    cursor.execute(select_stmt1)
    row = cursor.fetchone()
    res_array = []
    while row:
        res_array.append(row[0])
        row = cursor.fetchone()
    return(res_array)

In [121]:
filen="C:\\trial\\2018\\may\\2 MAY\\lvs\\AAP-McClintock.xlsx"
print(filen)
path_list=formating(filen)
entry = input("If this is the first time you are working with this ticker, enter [Y] else, enter [N]")
if(entry is 'Y'):      
    ticker_name = input("enter the ticker code")
    ticker_name = ticker_name.lower()
    if ticker_name in ticker_list:
        print("The ticker dictionary is already built")
        csv_file = ticker_name +"_matches.csv" 
    else:
        ticker_list.append(ticker_name.lower())
        ticker_dictionary_build(ticker_name)
        csv_file = ticker_name +"_matches.csv" 
ticker_dictionary = pd.read_csv(csv_file, error_bad_lines=False)
val=function1(filen,path_list)
if val==1:
    print("FAULT")
elif val==2:
    print("fault with sheet name")

C:\trial\2018\may\2 MAY\lvs\AAP-McClintock.xlsx
['2018', 'may', '2 MAY', 'AAP', 'McClintock']
If this is the first time you are working with this ticker, enter [Y] else, enter [N]Y
enter the ticker codelvs
enter the name of the excel sheet with the database metrics for this particular tickerlvs_actual_metric.csv
enter the name of the excel sheet with the analyst metrics for this particular tickertest1.csv
['ModelCover', 'Model', 'Quarterly Review', 'Segment', 'Share Repurchase', 'Metrics', 'Disclosures', 'Leases', 'Foresight']
['Model']
sheet to be checked
7
excel


'C:\\trial\\2018\\may\\2 MAY\\lvs\\AAP-McClintock.xlsx'

Model
This is excel data


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115
0,Financial Statements,1Q98,2Q98,3Q98,4Q98,1998.0,1Q99,2Q99,3Q99,4Q99,1999.000,1Q00,2Q00,3Q00,4Q00,2000.000,1Q01,2Q01,3Q01,4Q01,2001.000000,1Q02,2Q02,3Q02,4Q02,2002.000000,1Q03,2Q03,3Q03,4Q03,2003.000000,1Q04,2Q04,3Q04,4Q04,2004.000000,1Q05,2Q05,3Q05,4Q05,2005.000000,1Q06,2Q06,3Q06,4Q06,2006.000000,1Q07,2Q07,3Q07,4Q07,2007.000000,1Q08,2Q08,3Q08,4Q08,2008.000000,1Q09,2Q09,3Q09,4Q09,2009.000000,1Q10,2Q10,3Q10,4Q10,2010.000000,1Q11,2Q11,3Q11,4Q11,2011.000000,1Q12,2Q12,3Q12,4Q12,2012.000000,1Q13,2Q13,3Q13,4Q13,2013.000000,1Q14,2Q14,3Q14,4Q14,2014.000000,1Q15,2Q15,3Q15,4Q15,2015,1Q16,2Q16,3Q16,4Q16,2016,1Q17,2Q17,3Q17,4Q17,2017.000000,1Q18,2Q18E,3Q18E,4Q18E,2018E,1Q19E,2Q19E,3Q19E,4Q19E,2019E,1Q20E,2Q20E,3Q20E,4Q20E,2020E
1,Income Statement ($ mm),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Restated/Adjusted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,53.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,53.000000,A,A,A,A,A,A,A,A,A,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Net Sales,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,0.000,729.359,607.478,598.793,582.009,2517.639000,1004.09,792.717,788.662,689.195,3274.661000,1005.97,827.348,839.101,758.263,3430.680000,1122.92,908.412,890.161,848.806,3770.297000,1258.36,1023.15,1019.74,963.725,4264.971000,1393.01,1107.86,1099.49,1016.15,4616.503000,1468.12,1169.86,1158.04,1048.38,4844.404000,1526.13,1235.78,1187.95,1103.56,5053.427000,1683.64,1322.84,1262.58,1143.57,5412.623000,1830.61,1417.96,1406.51,1270.13,5925.203000,1898.06,1479.84,1464.99,1327.57,6170.462000,1957.29,1460.98,1457.53,1329.2,6205.003000,2015.3,1549.55,1520.14,1408.81,6493.814000,2969.5,2347.7,2289.46,2086.82,9693.475000,3038.23,2370.04,2295.2,2033.55,9737.02,2979.78,2256.16,2248.9,2082.89,9567.72,2890.84,2263.73,2182.23,2036.99,9373.784000,2873.85,2237.43,2176.51,2011.56,9299.34,2863.58,2229.96,2169.76,2005.31,9268.61,2897.63,2256.46,2195.53,2029.12,9378.74
4,% Δ y/y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.37667,0.304931,0.317086,0.184166,0.300687,0.00187334,0.0436865,0.0639552,0.100215,0.047644,0.116256,0.0979805,0.0608508,0.119408,0.098994,0.12062,0.126302,0.145564,0.135389,0.131203,0.107001,0.0827946,0.0782065,0.0543983,0.082423,0.0539192,0.0559657,0.0532585,0.0317197,0.049367,0.0395145,0.0563521,0.0258272,0.0526316,0.043147,0.103205,0.0704501,0.0628174,0.0362527,0.071080,0.0872932,0.0718996,0.114001,0.110674,0.094701,0.0368495,0.0436424,0.0415759,0.0452253,0.041393,0.031205,-0.0127419,-0.00509287,0.00122705,0.005598,0.0296389,0.0606236,0.0429611,0.0598946,0.046545,0.473474,0.51508,0.506078,0.481263,0.492724,0.0231467,0.00951571,0.0025102,-0.0255307,0.00449199,-0.0192398,-0.0480507,-0.0201738,0.024266,-0.0173866,-0.0298479,0.00335615,-0.0296443,-0.0220391,-0.020270,-0.00587719,-0.0116165,-0.00262323,-0.0124841,-0.0079414,-0.00357212,-0.00333753,-0.00310198,-0.0031068,-0.00330498,0.0118884,0.0118835,0.0118787,0.0118738,0.0118818

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102
0,Financial Statements,4Q00,2000.000,1Q01,2Q01,3Q01,4Q01,2001.000000,1Q02,2Q02,3Q02,4Q02,2002.000000,1Q03,2Q03,3Q03,4Q03,2003.000000,1Q04,2Q04,3Q04,4Q04,2004.000000,1Q05,2Q05,3Q05,4Q05,2005.000000,1Q06,2Q06,3Q06,4Q06,2006.000000,1Q07,2Q07,3Q07,4Q07,2007.000000,1Q08,2Q08,3Q08,4Q08,2008.000000,1Q09,2Q09,3Q09,4Q09,2009.000000,1Q10,2Q10,3Q10,4Q10,2010.000000,1Q11,2Q11,3Q11,4Q11,2011.000000,1Q12,2Q12,3Q12,4Q12,2012.000000,1Q13,2Q13,3Q13,4Q13,2013.000000,1Q14,2Q14,3Q14,4Q14,2014.000000,1Q15,2Q15,3Q15,4Q15,2015,1Q16,2Q16,3Q16,4Q16,2016,1Q17,2Q17,3Q17,4Q17,2017.000000,1Q18,2Q18E,3Q18E,4Q18E,2018E,1Q19E,2Q19E,3Q19E,4Q19E,2019E,1Q20E,2Q20E,3Q20E,4Q20E,2020E
1,Restated/Adjusted,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,53.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,53.000000,A,A,A,A,A,A,A,A,A,A,A,A,A,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Net Sales,NaN,NaN,729.359,607.478,598.793,582.009,2517.639000,1004.09,792.717,788.662,689.195,3274.661000,1005.97,827.348,839.101,758.263,3430.680000,1122.92,908.412,890.161,848.806,3770.297000,1258.36,1023.15,1019.74,963.725,4264.971000,1393.01,1107.86,1099.49,1016.15,4616.503000,1468.12,1169.86,1158.04,1048.38,4844.404000,1526.13,1235.78,1187.95,1103.56,5053.427000,1683.64,1322.84,1262.58,1143.57,5412.623000,1830.61,1417.96,1406.51,1270.13,5925.203000,1898.06,1479.84,1464.99,1327.57,6170.462000,1957.29,1460.98,1457.53,1329.2,6205.003000,2015.3,1549.55,1520.14,1408.81,6493.814000,2969.5,2347.7,2289.46,2086.82,9693.475000,3038.23,2370.04,2295.2,2033.55,9737.02,2979.78,2256.16,2248.9,2082.89,9567.72,2890.84,2263.73,2182.23,2036.99,9373.784000,2873.85,2237.43,2176.51,2011.56,9299.34,2863.58,2229.96,2169.76,2005.31,9268.61,2897.63,2256.46,2195.53,2029.12,9378.74
3,% Δ y/y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.37667,0.304931,0.317086,0.184166,0.300687,0.00187334,0.0436865,0.0639552,0.100215,0.047644,0.116256,0.0979805,0.0608508,0.119408,0.098994,0.12062,0.126302,0.145564,0.135389,0.131203,0.107001,0.0827946,0.0782065,0.0543983,0.082423,0.0539192,0.0559657,0.0532585,0.0317197,0.049367,0.0395145,0.0563521,0.0258272,0.0526316,0.043147,0.103205,0.0704501,0.0628174,0.0362527,0.071080,0.0872932,0.0718996,0.114001,0.110674,0.094701,0.0368495,0.0436424,0.0415759,0.0452253,0.041393,0.031205,-0.0127419,-0.00509287,0.00122705,0.005598,0.0296389,0.0606236,0.0429611,0.0598946,0.046545,0.473474,0.51508,0.506078,0.481263,0.492724,0.0231467,0.00951571,0.0025102,-0.0255307,0.00449199,-0.0192398,-0.0480507,-0.0201738,0.024266,-0.0173866,-0.0298479,0.00335615,-0.0296443,-0.0220391,-0.020270,-0.00587719,-0.0116165,-0.00262323,-0.0124841,-0.0079414,-0.00357212,-0.00333753,-0.00310198,-0.0031068,-0.00330498,0.0118884,0.0118835,0.0118787,0.0118738,0.0118818
4,COGS,NaN,NaN,417.909,350.25,342.059,331.395,1441.613000,567.579,443.703,439,378.628,1828.910000,541.979,447.874,452.173,412.898,1854.924000,602.02,486.11,473.646,455.15,2016.926000,657.433,541.096,538.321,513.643,2250.493000,727.842,580.498,569.28,537.719,2415.339000,758.717,606.998,602.93,554.79,2523.435000,801.278,649.501,625.777,583.172,2659.728000,861.648,670.194,641.117,595.438,2768.397000,919.829,702.668,698.726,642.645,2963.868000,939.862,743.991,740.485,676.834,3101.172000,976.619,732.125,732.177,666.046,3106.967000,1007.1,770.33,757.204,707.036,3241.668000,1616.38,1285.59,1255.01,1150.66,5307.642000,1644.31,1282.75,1262.82,1124.37,5314.25,1629.89,1245.9,1260.65,1175.33,5311.76,1620.15,1270.64,1234.53,1163.42,5288.735000,1601.56,1259.67,1236.26,115

103


,q1,q2,q3,q4,tl,Ex
2000,Nan,Nan,Nan,"[0, 1]","[0, 2]",Nan
2001,"[2, 3]","[2, 4]","[2, 5]","[2, 6]","[2, 7]",Nan
2002,"[2, 8]","[2, 9]","[2, 10]","[2, 11]","[2, 12]",Nan
2003,"[2, 13]","[2, 14]","[2, 15]","[2, 16]","[1, 17]",Nan
2004,"[2, 18]","[2, 19]","[2, 20]","[2, 21]","[2, 22]",Nan
2005,"[2, 23]","[2, 24]","[2, 25]","[2, 26]","[2, 27]",Nan
2006,"[2, 28]","[2, 29]","[2, 30]","[2, 31]","[2, 32]",Nan
2007,"[2, 33]","[2, 34]","[2, 35]","[2, 36]","[2, 37]",Nan
2008,"[2, 38]","[2, 39]","[2, 40]","[1, 41]","[1, 42]",Nan
2009,"[2, 43]","[2, 44]","[2, 45]","[2, 46]","[2, 47]",Nan


OperationalError: ('08S01', '[08S01] [Microsoft][ODBC Driver 17 for SQL Server]TCP Provider: An existing connection was forcibly closed by the remote host.\r\n (10054) (SQLExecDirectW); [08S01] [Microsoft][ODBC Driver 17 for SQL Server]Communication link failure (10054)')

In [123]:
ticker_dictionary.head()

,Unnamed: 0,trial,possible,matches
0,0,FCF & Balance Sheet Model,"['Accounting Equation Definition ', 'Accrued E...","['Accounts Payable', 'Accounts Receivable', 'A..."
1,1,Free Cash Flow Model,"['Asset Sales ', 'Asset Valuation ', 'Cash Flo...","['Adjusted EBITDA', 'Free Cash Flow', 'Operati..."
2,2,GAAP Net Income,['S&P Core Earnings '],"['Net Income', 'RevPar', 'Slots']"
3,3,D&A,"['1%/10 Net 30 ', '100% Equities Strategy ', '...",[]
4,4,FAS 123,"['2-1 Buydown ', '3D Printing ', '419(e) Welfa...","['Accounts Payable', 'Accounts Receivable', 'A..."


In [50]:
check_list

['Q1',
 'Q2',
 'Q3',
 'Q4',
 '1Q',
 '2Q',
 '3Q',
 '4Q',
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023,
 2024,
 2025,
 2026,
 2027,
 2028,
 2029,
 2030,
 2031,
 2032,
 2033,
 2034,
 2035,
 'FY',
 'CY']

In [124]:
ticker_dictionary[ticker_dictionary['trial']=="Free Cash Flow Model"].index.item()

1